# Simple Go-To-Goal for Cerus

The following code implements a simple go-to-goal behavior for Cerus. It uses a closed feedback loop to continuously asses Cerus' state (position and heading) in the world using data from two wheel encoders. It subsequently calculates the error between a given goal location and its current pose and will attempt to minimize the error until it reaches the goal location. A P-regulator (see PID regulator) function uses the error as an input and outputs the angular velocity for the Arduino and motor controllers that drive the robot. 

All models used in this program are adapted from Georgia Tech's "Control of Mobile Robots" by Dr. Magnus Egerstedt.    

In [1]:
#Import useful libraries
import serial
import time
import math

We first define our goal location. Units are metric, real-world coordinates in an X/Y coordinate system

In [2]:
goal_x = 0.5 #Goal X coordinate in meters
goal_y = -0.5 #Goal Y coordinate in meters
atGoal = False
constVel = 0.5 #To simplify this program, we're using a constant linear velocity to reach our goal

We use pySerial to read encoder data from the Arduino and send move commands to it:

In [3]:
#Opening a serial port on the Arduino resets it, so our encoder count is also reset to 0,0
ser = serial.Serial('COM3', 115200) #replace 'COM3' with the appropriate serial port on your device
time.sleep(1)

The Cerus class keeps track of all the important robot parameters.

In [4]:
class Cerus():
    def __init__(self, pose_x, pose_y, pose_phi, R_wheel, N_ticks, L_track):
        self.pose_x = pose_x #X position
        self.pose_y = pose_y #Y position
        self.pose_phi = pose_phi #Heading
        self.R_wheel = R_wheel #wheel radius in meters
        self.N_ticks = N_ticks #encoder ticks per wheel revolution
        self.L_track = L_track #wheel track in meters        

#Create a Cerus instance and initialize it to a 0,0,0 world position and with some physical dimensions 
cerus = Cerus(0,0,0,0.03,500,0.23)

Pose calculation allows us track where our robot is in space as it moves. The pose is often also referred to as the 'state'.

In [5]:
def calculatePose(deltaTicks):
    
    #Calculate the centerline distance moved
    distanceLeft = 2 * math.pi * cerus.R_wheel * (deltaTicks[0] / cerus.N_ticks)
    distanceRight = 2 * math.pi * cerus.R_wheel * (deltaTicks[1] / cerus.N_ticks)
    distanceCenter = (distanceLeft + distanceRight) / 2
            
    #Update the position and heading
    cerus.pose_x = round((cerus.pose_x + distanceCenter * math.cos(cerus.pose_phi)), 4)
    cerus.pose_y = round((cerus.pose_y + distanceCenter * math.sin(cerus.pose_phi)), 4)
    cerus.pose_phi = round((cerus.pose_phi + ((distanceRight - distanceLeft) / cerus.L_track)), 4)

Additionally, we want to keep track of how far we're off the goal point defined initially.

In [6]:
#Calculate the error between Cerus' heading and the goal point
def calculateError():
    
    phi_desired = math.atan2((goal_y - cerus.pose_y),(goal_x - cerus.pose_x))    
    
    temp = phi_desired - cerus.pose_phi     
    
    error_heading = round((math.atan2(math.sin(temp), math.cos(temp))), 4) #ensure that error is within [-pi, pi]
    error_x = round((goal_x - cerus.pose_x), 4)
    error_y = round((goal_y - cerus.pose_y), 4)
    
    print("The heading error is: ", error_heading)
    print("The X error is: ", error_x)
    print("The Y error is: ", error_y)
    return error_x, error_y, error_heading

Finally, we want to read our encoders, calculate our pose, calculate the goal error and issue a move command if necessary.

In [7]:
def moveRobot(): 
    
    global atGoal
    atGoal = False
    
    #Everytime we call this function, we read two sets of encoder values and evaluate the delta  
    data = ["0,0","0,0"]
    i = 0    
       
    while i < 2 and atGoal == False:
        if ser.inWaiting():                 
            temp = ser.readline()
            data[i] = temp.decode()
            
            leftValOld, rightValOld = formatData(data[0])
            leftValNew, rightValNew = formatData(data[1])
            i += 1
                   
        
    #From these values we can calculate the momentary encoder values for both sides of the robot
    leftDelta = leftValNew - leftValOld
    rightDelta = rightValNew - rightValOld
    deltaTicks = [leftDelta, rightDelta]    
            
    #Calculate current pose
    calculatePose(deltaTicks)        
        
    #Calculate the current pose to goal error
    error_x, error_y, error_heading = calculateError()
            
    #If we're within 5 cm of the goal
    if error_x <= 0.05 and error_y <= 0.05:
        twist(0.0,0.0)   
        atGoal = True        
        
    #Otherwise keep driving, using P-controller to adjust angular velocity
    else:
        omega = - (2 * error_heading)
        twist(constVel, omega)        

The functions below are helpers and will be called through our main loop.

In [8]:
#Functions to read and format encoder data received from the Serial port
def formatData(data):
    delimiter = "x"
    leftVal = ""
    rightVal = ""
    
    for i in range(len(data)):        
        if data[i] == ",":
            delimiter = ","            
        elif delimiter != ",":
            leftVal += data[i]            
        elif delimiter == ",":
            rightVal += data[i]    
       
    leftVal, rightVal = int(leftVal), int(rightVal)    
    return leftVal, rightVal

In [9]:
#Create a function that sends a movement command to the Arduino
def twist(linearVelocity, angularVelocity):       
        
        command = f"<{linearVelocity},{angularVelocity}>"
        ser.write(str.encode(command))

This is the main part for our program that will loop over and over until Cerus has reached its goal. For our simple go-to-goal behavior, we will drive the robot at a constant speed and only adjust our heading so that we reach the goal location.

__WARNING: This will move the robot!__

In [10]:
while not atGoal:
    
    moveRobot()    
    
print("Robot at goal position.")

Delta Ticks: 0 0
The heading error is:  -0.7854
The X error is:  0.5
The Y error is:  -0.5
Delta Ticks: 74 -71
The heading error is:  -0.5483
The X error is:  0.4994
The Y error is:  -0.5
Delta Ticks: 102 -73
The heading error is:  -0.2655
The X error is:  0.4941
The Y error is:  -0.4987
Delta Ticks: 101 -18
The heading error is:  -0.0764
The X error is:  0.4806
The Y error is:  -0.4909
Delta Ticks: 79 24
The heading error is:  0.0116
The X error is:  0.466
The Y error is:  -0.4781
Delta Ticks: 53 46
The heading error is:  0.0234
The X error is:  0.4531
The Y error is:  -0.4646
Delta Ticks: 48 53
The heading error is:  0.0158
The X error is:  0.4401
The Y error is:  -0.4507
Delta Ticks: 47 53
The heading error is:  0.0064
The X error is:  0.4271
The Y error is:  -0.437
Delta Ticks: 50 51
The heading error is:  0.0051
The X error is:  0.4139
The Y error is:  -0.4233
Delta Ticks: 50 50
The heading error is:  0.0052
The X error is:  0.4008
The Y error is:  -0.4098
Delta Ticks: 50 51
The h

In [11]:
# Close the serial connection when done
ser.close()